In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
import argparse
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
import os
print(os.getcwd())
print("Ok")

/Users/zhangzhen
Ok


In [39]:
max_step = 1000  # 最大迭代次数
learning_rate = 0.001   # 学习率
dropout = 0.9   # dropout时随机保留神经元的比例

data_dir = '/Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/'  # 样本数据存储的路径
log_dir = '/Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/tensorboard/log'  # 输出日志保存的路径

mnist = input_data.read_data_sets(data_dir, one_hot=True)

tf.reset_default_graph()
sess = tf.InteractiveSession()

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')


with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])  # [None, 28, 28, 1]
    tf.summary.image('input', image_shaped_input, 10)


def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
    
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    
    # 用直方图记录参数的分布
    tf.summary.histogram('histogram', var)


def nn_layer(input_tensor, dim_input, dim_out, layer_name, activation=tf.nn.relu):
    
    with tf.name_scope(layer_name):
        
        with tf.name_scope('weights'):
            weights = weight_variable([dim_input, dim_out])
            variable_summaries(weights)
        
        with tf.name_scope('biases'):
            biases = bias_variable([dim_out])
            variable_summaries(biases)
            
        with tf.name_scope("linear_compute"):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('linear', preactivate)
        
        output = activation(preactivate, name='activation')
        tf.summary.histogram('output', output)
        
        return output


hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 500, 10, 'layer2', activation=tf.identity)

with tf.name_scope('loss'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)

tf.summary.scalar('loss', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('accuracy', accuracy)


# summaries 合并
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')

tf.global_variables_initializer().run()


def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
        xs, ys = mnist.train.next_batch(100)
        # print(xs.shape, ys.shape)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y_: ys, keep_prob: k}


for i in range(max_step):
    # 记录测试集的summary与accuracy
    if i % 10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:
        if i % 100 == 99:  # Record execution stats
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            
            summary, _ = sess.run([merged, train_step], 
                                  feed_dict=feed_dict(True), 
                                  options=run_options, 
                                  run_metadata=run_metadata)
            
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            print('Adding run metadata for', i)
        else:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)
        
train_writer.close()
test_writer.close()

Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/train-images-idx3-ubyte.gz


Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/t10k-images-idx3-ubyte.gz


Extracting /Users/zhangzhen/gitRepository/AnalyticsVidhya/_tensorflow/MNIST_data/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Accuracy at step 0: 0.0466


...

Accuracy at step 960: 0.967


Accuracy at step 970: 0.9679


Accuracy at step 980: 0.9647


Accuracy at step 990: 0.9697


Adding run metadata for 999
